<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=cc17907500869ef1d5e9f18833d8fbfeebb461454933d3fceae7992d4da04a4c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-24 11:25:41
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 16.97 K (0.12%)
Current PnL: -25.11 L (-16.22%)
CY Booked + Current PnL: -10.82 L (-6.99%)
-------------------
Total profit:  1.66 L
Total loss:  -26.77 L
-------------------
Total Booked + Current PnL: 16.16 L (12.64%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.07%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 89.74 L (63.23%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.96,10.52,8.30,19.69,17589.0,20178.0,211920.0,-0.25,60.0,X-MC,2.51,78.0,1.15,1.50,18.04,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.04,-19.16,23.81,0.09,19398.0,-19308.0,81469.0,89.11,38.0,M-SC,2.62,253.0,-1.00,0.58,7.03,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.25,-0.86,23.10,22.05,22439.0,-840.0,97140.0,-14.63,58.0,H-SC,6.67,164.0,-0.04,0.69,41.20,XR,ATH,FINANCE
43,ITC,452.00,0.11,-0.05,10.84,10.78,26037.0,-124.0,240194.0,-40.34,58.0,X-LC,2.01,5.0,-0.00,1.70,5.18,X40,NTT,FMCG
52,NESTLEIND,1377.00,0.29,11.10,9.27,21.40,27336.0,29461.0,294887.0,3.02,57.0,X-LC,5.53,12.0,1.08,2.09,18.48,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,3.51,-1.92,35.01,32.42,87180.0,-4881.0,249015.0,-19.39,50.0,X-SC,10.68,85.0,-0.06,1.76,12.89,X40,ATH,FMCG
47,KANSAINER,340.00,2.69,-23.14,47.63,13.47,98723.0,-62397.0,207270.0,-68.31,51.0,H-SC,6.67,158.0,-0.63,1.47,5.39,XY24,NTT,PAINTS
0,5PAISA,593.00,2.60,-38.31,83.02,12.90,111898.0,-83712.0,134784.0,98.32,55.0,H-SC,12.66,161.0,-0.75,0.95,12.07,OX40N,NTT,FINANCE
74,TITAGARH,1548.00,2.51,-19.22,80.41,45.74,157310.0,-46542.0,195635.0,7.63,63.0,H-SC,6.40,174.0,-0.30,1.38,27.24,XY24,NTT,ENGINEERING
63,SFL,1287.00,2.51,-41.11,115.04,26.63,177637.0,-107805.0,154413.0,16.95,49.0,M-SC,12.32,239.0,-0.61,1.09,5.71,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,671.64,-1.43,-4.08,41.44,35.67,125544.0,-12888.0,302954.0,-15.46,52.0,X-LC,3.47,4.0,-0.10,2.14,9.21,X40N,ATH,FMCG
57,RAJOOENG,143.10,-1.29,-38.68,127.68,39.61,80247.0,-39650.0,62850.0,-58.53,31.0,H-SC,31.09,135.0,-0.49,0.44,0.00,AR,ATH,MISC
83,WHIRLPOOL,2270.00,-1.17,-25.36,143.75,81.92,123165.0,-29118.0,85680.0,-56.34,31.0,M-SC,21.76,236.0,-0.24,0.61,2.21,XR,NTT,DURABLES
6,ASIANTILES,137.00,-1.08,-3.78,88.81,81.67,80164.0,-3545.0,90265.0,8127.78,69.0,L-SC,12.83,271.0,-0.04,0.64,74.80,XR,NTT,CERAMICS
51,MEDANTA,1486.00,-1.07,-1.87,26.44,24.08,32939.0,-2368.0,124582.0,-9.82,45.0,X-SC,6.46,89.0,-0.07,0.88,17.38,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,1.66,0.10,19.11,19.23,28781.0,148.0,150608.0,16.96,77.0,L-LC,2.52,179.0,0.01,1.07,21.4,XY25,ATH,MINING
50,MASFIN,398.61,0.25,-0.86,23.10,22.05,22439.0,-840.0,97140.0,-14.63,58.0,H-SC,6.67,164.0,-0.04,0.69,41.2,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.38,-2.54,113.34,107.91,163082.0,-3757.0,143887.0,-20.55,52.0,M-SC,11.86,216.0,-0.02,1.02,7.29,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.08,-17.01,20.52,0.02,29727.0,-29690.0,144869.0,120.98,40.0,M-SC,11.39,234.0,-1.00,1.03,26.01,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.04,-19.16,23.81,0.09,19398.0,-19308.0,81469.0,89.11,38.0,M-SC,2.62,253.0,-1.00,0.58,7.03,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.64,-22.26,69.84,32.03,71576.0,-29349.0,102486.0,-15.87,60.0,H-SC,8.96,148.0,-0.41,0.73,14.79,OX40N,NTT,IT
47,KANSAINER,340.00,2.69,-23.14,47.63,13.47,98723.0,-62397.0,207270.0,-68.31,51.0,H-SC,6.67,158.0,-0.63,1.47,5.39,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,1.66,0.10,19.11,19.23,28781.0,148.0,150608.0,16.96,77.0,L-LC,2.52,179.0,0.01,1.07,21.40,XY25,ATH,MINING
84,WIPRO,420.00,-1.02,10.18,56.34,72.27,105190.0,17257.0,186705.0,-5.63,68.0,M-LC,3.89,99.0,0.16,1.32,16.99,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.02,10.18,56.34,72.27,105190.0,17257.0,186705.0,-5.63,68.0,M-LC,3.89,99.0,0.16,1.32,16.99,XR,NTT,IT
25,FINCABLES,1641.55,-0.38,-2.54,113.34,107.91,163082.0,-3757.0,143887.0,-20.55,52.0,M-SC,11.86,216.0,-0.02,1.02,7.29,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.67,-3.54,114.32,106.72,136000.0,-4372.0,118964.0,-52.15,42.0,H-SC,4.48,139.0,-0.03,0.84,19.00,AR,ATH,MISC
6,ASIANTILES,137.00,-1.08,-3.78,88.81,81.67,80164.0,-3545.0,90265.0,8127.78,69.0,L-SC,12.83,271.0,-0.04,0.64,74.80,XR,NTT,CERAMICS
50,MASFIN,398.61,0.25,-0.86,23.10,22.05,22439.0,-840.0,97140.0,-14.63,58.0,H-SC,6.67,164.0,-0.04,0.69,41.20,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.55,-18.35,90.41,55.47,179597.0,-44639.0,198647.0,-27.60,30.0,X-MC,15.61,64.0,-0.25,1.41,1.78,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.18,-18.23,75.27,43.32,100245.0,-29692.0,133180.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.26,X40N,ATH,ELECTRICAL
8,AWL,485.00,-0.21,-24.41,103.06,53.50,235082.0,-73649.0,228102.0,-63.37,32.0,X-MC,5.90,58.0,-0.31,1.61,1.31,XY24,NTT,FMCG
3,ACC,3906.00,-0.53,-26.67,123.85,64.15,216106.0,-63461.0,174490.0,-57.19,34.0,X-SC,4.20,82.0,-0.29,1.24,0.00,XY24,BTT,CEMENT
20,DABUR,735.00,-0.19,-3.77,49.33,43.70,117759.0,-9351.0,238717.0,-11.62,34.0,X-MC,2.11,72.0,-0.08,1.69,10.71,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.34,-2.75,24.56,21.14,54300.0,-6249.0,221090.0,-9.65,47.0,X-MC,1.03,74.0,-0.12,1.56,24.62,XY24,NTT,PAINTS
15,CAMS,950.0,1.22,0.83,23.46,24.48,59573.0,2079.0,253935.0,-80.87,52.0,X-SC,1.29,86.0,0.03,1.80,24.15,X40N,NTT,MISC
78,UNITDSPR,1644.0,-0.84,5.02,14.99,20.77,36862.0,11764.0,245908.0,-4.14,51.0,X-MC,1.97,62.0,0.32,1.74,12.29,X40N,NTT,BREWERIES
43,ITC,452.0,0.11,-0.05,10.84,10.78,26037.0,-124.0,240194.0,-40.34,58.0,X-LC,2.01,5.0,-0.00,1.70,5.18,X40,NTT,FMCG
20,DABUR,735.0,-0.19,-3.77,49.33,43.70,117759.0,-9351.0,238717.0,-11.62,34.0,X-MC,2.11,72.0,-0.08,1.69,10.71,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.53,-26.67,123.85,64.15,216106.0,-63461.0,174490.0,-57.19,34.0,X-SC,4.20,82.0,-0.29,1.24,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,0.13,-19.92,76.71,41.51,161781.0,-52465.0,210900.0,-4.54,41.0,X-MC,7.85,61.0,-0.32,1.49,1.03,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.13,-38.77,120.82,35.20,95187.0,-49886.0,78784.0,3.25,35.0,X-SC,17.01,92.0,-0.52,0.56,1.05,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.21,-24.41,103.06,53.50,235082.0,-73649.0,228102.0,-63.37,32.0,X-MC,5.90,58.0,-0.31,1.61,1.31,XY24,NTT,FMCG
9,BAJAJHFL,181.50,-0.55,-18.35,90.41,55.47,179597.0,-44639.0,198647.0,-27.60,30.0,X-MC,15.61,64.0,-0.25,1.41,1.78,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.96,-27.56,97.19,42.84,45766.0,-17917.0,47089.0,-52.62,53.0,X-SC,33.79,83.0,-0.39,0.33,6.54,XY24,NTT,MISC
59,RELAXO,1176.00,0.97,-47.24,188.70,52.31,144510.0,-68578.0,76582.0,-43.86,47.0,X-SC,7.32,91.0,-0.47,0.54,2.51,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.13,-38.77,120.82,35.20,95187.0,-49886.0,78784.0,3.25,35.0,X-SC,17.01,92.0,-0.52,0.56,1.05,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.07,-1.87,26.44,24.08,32939.0,-2368.0,124582.0,-9.82,45.0,X-SC,6.46,89.0,-0.07,0.88,17.38,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.18,-18.23,75.27,43.32,100245.0,-29692.0,133180.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.26,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.03,-8.96,32.59,20.71,112222.0,-33893.0,344344.0,-21.68,70.0,X-LC,3.21,1.0,-0.30,2.44,15.06,X40,ATH,IT
41,INFY,1972.00,-0.55,12.95,18.85,34.24,66618.0,40522.0,353410.0,-12.06,66.0,X-LC,6.14,2.0,0.61,2.50,22.31,X40,NTT,IT
75,TMPV,600.00,-0.95,-26.26,66.76,22.98,111213.0,-59311.0,166587.0,-23.08,50.0,X-LC,6.40,3.0,-0.53,1.18,4.78,XY24,NTT,AUTO
81,VBL,671.64,-1.43,-4.08,41.44,35.67,125544.0,-12888.0,302954.0,-15.46,52.0,X-LC,3.47,4.0,-0.10,2.14,9.21,X40N,ATH,FMCG
43,ITC,452.00,0.11,-0.05,10.84,10.78,26037.0,-124.0,240194.0,-40.34,58.0,X-LC,2.01,5.0,-0.00,1.70,5.18,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.16,-37.15,111.44,32.89,54075.0,-28682.0,48524.0,-699.18,55.0,L-MC,6.81,259.0,-0.53,0.34,33.63,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-0.33,-57.06,133.03,0.07,78364.0,-78270.0,58907.0,2059.09,70.0,L-SC,4.04,268.0,-1.00,0.42,46.50,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.08,-3.78,88.81,81.67,80164.0,-3545.0,90265.0,8127.78,69.0,L-SC,12.83,271.0,-0.04,0.64,74.80,XR,NTT,CERAMICS
50,MASFIN,398.61,0.25,-0.86,23.10,22.05,22439.0,-840.0,97140.0,-14.63,58.0,H-SC,6.67,164.0,-0.04,0.69,41.20,XR,ATH,FINANCE
13,BSOFT,831.70,-0.98,-18.16,84.25,50.79,96978.0,-25546.0,115107.0,5.14,70.0,H-SC,8.43,151.0,-0.26,0.81,34.02,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.98,-18.16,84.25,50.79,96978.0,-25546.0,115107.0,5.14,70.0,H-SC,8.43,151.0,-0.26,0.81,34.02,XR,ATH,IT
56,RAJESHEXPO,518.00,-0.33,-57.06,133.03,0.07,78364.0,-78270.0,58907.0,2059.09,70.0,L-SC,4.04,268.0,-1.00,0.42,46.50,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.08,-3.78,88.81,81.67,80164.0,-3545.0,90265.0,8127.78,69.0,L-SC,12.83,271.0,-0.04,0.64,74.80,XR,NTT,CERAMICS
61,ROUTE,2227.21,-0.94,-46.24,214.47,69.05,153417.0,-61533.0,71533.0,-56.49,53.0,H-SC,20.63,140.0,-0.40,0.51,10.32,SR,ATH,IT
17,COALINDIA,484.83,1.66,0.10,19.11,19.23,28781.0,148.0,150608.0,16.96,77.0,L-LC,2.52,179.0,0.01,1.07,21.40,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.35
1,25,44.78
2,50,76.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.01
MC    30.71
LC    24.23
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      23.04
X40N     12.91
XR        9.91
XY25      9.16
AR        8.89
OX40N     7.54
SR        1.00
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.19
H-SC    23.87
X-LC    20.73
M-SC    11.55
X-SC     8.05
H-MC     4.79
L-SC     1.54
M-MC     1.39
M-LC     1.32
H-LC     1.11
L-LC     1.07
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.19,-7.31,41.82
IT,13.13,-13.33,70.31
FINANCE,9.58,-17.71,68.91
MISC,7.28,-26.07,77.64
ELECTRICAL,5.96,-11.44,52.26
PAINTS,5.80,-12.66,29.40
INSURANCE,4.73,-1.30,35.97
PHARMA,4.04,-2.91,35.30
AUTO,2.84,-30.47,75.71


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3139783.0,21
AR,1315418.0,10
XR,1259879.0,13
X40,966551.0,14
X40N,860351.0,9
OX40N,728790.0,10
XY25,347484.0,6
SR,279663.0,2
MH,76517.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3603825.0,25
M-SC,1426961.0,15
X-MC,1405735.0,16
X-LC,796268.0,11
X-SC,781506.0,8
H-MC,401880.0,3
L-SC,243626.0,3
M-LC,105190.0,1
H-LC,77451.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1254410.0      6
           AR         902345.0      5
M-SC       XY24       806698.0      6
H-SC       XR         760419.0      7
X-MC       X40        483118.0      7
           XY24       407141.0      3
           X40N       336106.0      4
H-SC       OX40N      330471.0      4
M-SC       OX40N      319955.0      5
X-SC       X40N       304328.0      3
X-LC       X40        301066.0      5
X-SC       XY24       294811.0      3
H-SC       SR         279663.0      2
X-LC       X40N       219917.0      2
H-MC       AR         210247.0      2
           XY24       191633.0      1
X-LC       XY24       185090.0      2
X-SC       X40        182367.0      2
X-MC       XY25       179370.0      2
M-SC       XR         174933.0      2
L-SC       XR         165262.0      2
M-SC       AR         125375.0      2
M-LC       XR         105190.0      1
X-LC       XY25        90195.0      2
L-SC       OX40N       78364.0      1
H-LC       AR          77451.0      1
H-SC       MH          76517.0      1
L-MC       XR          54075.0      1
M-MC       XY25        49138.0      1
L-LC       XY25        28781.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
